In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_teddynote import logging
logging.langsmith('Ch03-OutputParser')

LangSmith 추적을 시작합니다.
[프로젝트명]
Ch03-OutputParser


In [3]:
from langchain_teddynote.messages import stream_response

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

llm = ChatOpenAI(temperature=0, model_name='gpt-4o')

In [6]:
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

In [7]:
from itertools import chain
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    f"다음의 이메일 내용 중에서 중요한 내용을 추출해주세요 \n\n {email_conversation}"
)

llm = ChatOpenAI(temperature=0, model_name='gpt-4o')

chain = prompt | llm
answer = chain.stream({"email_conversation": email_conversation})
output = stream_response(answer, return_output=True)

**중요 내용 추출:**

1. **발신자:** 김철수 (바이크코퍼레이션 상무)
2. **수신자:** 이은채 (테디인터내셔널 대리)
3. **주제:** "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안
4. **요청 사항:**
   - ZENESIS 모델의 상세한 브로슈어 요청 (기술 사양, 배터리 성능, 디자인 정보 포함)
5. **미팅 제안:**
   - 날짜: 다음 주 화요일 (1월 15일)
   - 시간: 오전 10시
   - 장소: 테디인터내셔널 사무실

In [8]:
print(output)

**중요 내용 추출:**

1. **발신자:** 김철수 (바이크코퍼레이션 상무)
2. **수신자:** 이은채 (테디인터내셔널 대리)
3. **주제:** "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안
4. **요청 사항:**
   - ZENESIS 모델의 상세한 브로슈어 요청 (기술 사양, 배터리 성능, 디자인 정보 포함)
5. **미팅 제안:**
   - 날짜: 다음 주 화요일 (1월 15일)
   - 시간: 오전 10시
   - 장소: 테디인터내셔널 사무실


In [9]:
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")


# PydanticOutputParser 생성
parser = PydanticOutputParser(pydantic_object=EmailSummary)

In [10]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"title": "Person", "description": "메일을 보낸 사람", "type": "string"}, "email": {"title": "Email", "description": "메일을 보낸 사람의 이메일 주소", "type": "string"}, "subject": {"title": "Subject", "description": "메일 제목", "type": "string"}, "summary": {"title": "Summary", "description": "메일 본문을 요약한 텍스트", "type": "string"}, "date": {"title": "Date", "description": "메일 본문에 언급된 미팅 날짜와 시간", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}
```
